In [ ]:
import numpy as np

import adaptive_latents
from adaptive_latents import AnimationManager, Pipeline, CenteringTransformer, proSVD, Bubblewrap, VanillaOnlineRegressor, NumpyTimedDataSource, KernelSmoother, Concatenator
import adaptive_latents.input_sources.datasets as datasets
import adaptive_latents.plotting_functions as pf
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import copy


## Intro video

In [ ]:
d = datasets.Odoherty21Dataset()

tail_length = 5  # seconds

with AnimationManager(n_cols=2, n_rows=1, figsize=(10, 5)) as am:
    for current_t in np.linspace(100, 110, 51):
        am.axs[0,0].cla()

        n_columns = np.floor(tail_length / d.bin_width).astype(int)
        idx = np.nonzero(~(d.neural_data.t < current_t))[0][0]
        
        
        am.axs[0,0].imshow(d.neural_data.a[idx-n_columns:idx,0,:].T, aspect='auto', interpolation='none', extent=[current_t - tail_length, current_t, d.neural_data.a.shape[2], 0])



        old_lims = am.axs[0,1].axis()
        am.axs[0,1].cla()
        am.axs[0,1].axis('off')
        
        s = ((current_t - tail_length) < d.behavioral_data.t) & (d.behavioral_data.t < current_t)
        am.axs[0,1].plot(d.behavioral_data.a[s,0,0], d.behavioral_data.a[s,0,1])
        pf.use_bigger_lims(am.axs[0,1], old_lims)
        
        am.grab_frame()

## Regression video

In [ ]:
d = datasets.Temmar24uDataset()


p = Pipeline([
    proSVD(k=6, input_streams={0:'X'}, output_streams={0:0}),
    KernelSmoother(tau=0.2/d.bin_width, input_streams={0:'X'}, output_streams={0:0}),
    Concatenator(input_streams={0: 0, 1: 1}, output_streams={0:2, 1:2}),
    Bubblewrap(
        num=100,
        eps=1e-3,
        step=1,
        num_grad_q=3,
        input_streams={2:'X'}, 
        output_streams={2:2},
    ),
    VanillaOnlineRegressor(
        log_level=1,
        input_streams={2:'X', 3:'Y'},
        output_streams={2:2},
    ),
])

bw: Bubblewrap = p.steps[-2]
reg: VanillaOnlineRegressor = p.steps[-1]

streams = []
streams.append( (d.neural_data, 0) )
streams.append( (d.behavioral_data, 1) )
streams.append( (d.behavioral_data, 3) )
iterator = p.streaming_run_on(streams, return_output_stream=True)

In [ ]:
time_between_draws = .1
time_of_last_draw = 100
exit_time = 110
prediction = []
prediction_t = []
pbar = tqdm(total=exit_time)

with AnimationManager(n_cols=1, n_rows=1, figsize=(10, 5)) as am:
    ax = am.axs[0,0]
    ax.text(.01, .01, "\n".join([f"{k}: {v}" for k, v in p.get_params().items()]), transform=ax.transAxes, fontsize='xx-small', wrap=True)
    am.grab_frame()
    
    for output, stream in iterator:
        current_t = output.t
        pbar.update(current_t - pbar.n)

        if stream == 3 and current_t > time_of_last_draw - 10:
            prediction_offset = .1
            time_to_predict = bw.last_timepoint + prediction_offset
            output = reg.predict(bw.get_alpha_at_t(time_to_predict)) # todo: why doesn't transform work?
            prediction.append(output)
            prediction_t.append(time_to_predict)

        if current_t > time_of_last_draw + time_between_draws:
            old_axes = ax.axis()
            ax.cla()

            s = (current_t-5 < d.behavioral_data.t) & (d.behavioral_data.t < current_t)
            ax.plot(d.behavioral_data.t[s], d.behavioral_data.a[s,0,0])
            ax.plot(d.behavioral_data.t[s], d.behavioral_data.a[s,0,1])

            if len(prediction):
                pt = np.array(prediction_t)
                s = (current_t-5 < pt) & (pt < current_t)
                pred = np.squeeze(prediction)[s]
                ax.plot(pt[s], pred[:, 0], 'k')
                ax.plot(pt[s], pred[:, 1], 'k')

            pf.use_bigger_lims(ax, old_axes, x=False)
            am.grab_frame()
            time_of_last_draw = current_t

        if current_t > exit_time:
            break



In [ ]:

reg = VanillaOnlineRegressor(log_level=1)

reg.partial_fit_transform(np.array([0]),0)
reg.partial_fit_transform(np.array([0]),1)

reg.partial_fit_transform(np.array([0]),0)
reg.partial_fit_transform(np.array([0]),1)

reg.partial_fit_transform(np.array([0]),0)
reg.partial_fit_transform(np.array([0]),1)

reg.partial_fit_transform(np.array([0]),0)
reg.partial_fit_transform(np.array([0]),1)

reg.log
